In [1]:
import asyncio
import os
from openai import AsyncOpenAI
import pandas as pd
from jinja2 import Template
import json

In [7]:
with open("augmented_data_problem_queries.json") as f:
    data = json.load(f)

In [8]:
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

## Checking query tokens and duplicate queries

In [9]:
token_count = 0

In [10]:
check_duplicates = {}
duplicate_data = []

In [12]:
# query_count = 0
# for key, item in data.items():
#     index = 0
#     for query in item:
#         index += 1
#         token_count += len(query)
#         if check_duplicates.get(query):
#             duplicate_data.append({"key": key, "query" : query, "index": index})
#     query_count += index

In [13]:
print(f"Total Tokens of Queries: {token_count}\nLenght Duplicate Data: {len(duplicate_data)}\nTotal Count of Queries: {query_count}")

Total Tokens of Queries: 0
Lenght Duplicate Data: 0
Total Count of Queries: 0


# Duplicating the queries??

In [3]:
NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT_ORIGINAL = """
You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.
"""


NER_MANAGEMENT_LEVEL_TITLE_USER_PROMPT_ORIGINAL = """
Instructions:
1. Management Levels: Only return management levels that match the predefined set: ["Partners,"," "Founder or Co-founder," "Board of Directors," "CSuite/Chiefs," "Executive VP or Sr. VP," "General Manager," "Manager," "Head," "Senior Partner," "Junior Partner," "VP," "Director," "Senior (All Senior-Level Individual Contributors)," "Mid (All Mid-Level Individual Contributors)," "Junior (All Junior-Level Individual Contributors)"]. MANAGEMENT CAN ONLY BE FROM THIS PREDEFINED SET, Nothing ELSE.
2. Job Titles: Normalize the job title after extracting it from the text. For example, convert "ceo" to "Chief Executive Officer" and always include both the full title and its abbreviation (confirmed ones), e.g., "VP of Engineering" and "Vice President of Engineering." or "Chief Innovation Officer" and "CINO". ENSURE LOGICAL and EXACT job titles such as 'Architect' NOT 'Architect who is skilled in VR'. Job titles MUST BE CONCISE AND TO THE POINT and shouldn't include company names or region names. Do not change the title for normalization.
3. Response Format: Your response must be a dictionary with two keys: "management_levels" and "titles". Each key should have a list of management levels and titles respectively.
4. Identify the key phrases in a prompt. Key phrase is a title and its function, IF THE function is mentioned. If the function would be mentioned, it will be classified as a "Job Title". For example, "CEOs working in Automotive Industry and VP of Engineering of Microsoft" has the "CEOs" and "VP of Engineering" as Key Phrases. In this case ONLY VP CANNOT be considered a key phrase. ONLY CONSIDER THE KEY PHRASES MENTIONED, DO NOT ASSUME. Past and current designations dont matter.
5. Check whether the KEY PHRASE should be classified as a Job Title or a Management Level. IT SHOULD NEVER BE CLASSIFIED INTO BOTH. This is ESSENTIAL.
5. If a key phrase is classified as title, don't include it in the management levels. For example, if "VP of Engineering" is classified as title then don't include "VP" in management levels. Industry or company names will not be included in job titles.
6. If a key phrase is classified as a management level, don't include it in the title. For example, if "Vice Presidents" is classified as management level then don't include "Vice Presidents" in titles. A job title will ONLY be a title and its business function. No other DETAIL should be added. A Management Level cannot COME FROM WITHIN A JOB TITLE.
7. Remember: One instance of a key phrase should be considered for either management level or job title, not both. Each will fall either into management levels or job title but WILL NEVER FALL INTO BOTH. A KEY PHRASE CANNOT BE IN MANAGEMENT LEVELS AND TITLE, BOTH. A Management Level cannot COME FROM WITHIN A JOB TITLE. 
8. If the word 'executive' is mentioned, specific considerations should be taken into account.

Take a deep breath and understand.
Query: "Give me VPs working in Microsoft": # VPs is the KEY PHRASE. It will be evaluated as a whole key phrase ONLY. It cannot be classified into a management level and a title both.
Management Level Focus: In this query, "VPs" should refer to individuals at the management level of Vice President within Microsoft. This means you are asking for a list of people who occupy the VP rank across various departments or divisions within the company. VPs can cover the complete domain of of 'VP' in management level. The emphasis is on their standing in the organizational hierarchy, regardless of their specific job titles. A management title will only be selected if it covers the complete domain in the predefined set.
Job Title Focus: If you were asking about "VPs" in terms of job titles, you'd be interested in individuals whose specific title is "VP" of a certain business function, such as "VP of Marketing" (Marketing is a business function) or "VP of Engineering" (Engineering is a business function). If a function is clearly mentioned then it would be JOB TITLE. "VP of Microsoft" does NOT have a function (Microsoft is an organization) neither does "Automotive VPs" (Automotive can ONLY be an industry). Identify the BUSINESS functions accurately. Then they CANNOT come under management levels.
Output: {"management_levels": ["VP"], "titles": []}

Query: "The CFOs working in google or facebook": #CFOs is a KEY PHRASE. It will be evaluated as a whole key phrase ONLY. It cannot be classified into a management level and a title both.
Management Level Focus: In this query, "CFOs" does not cover the complete 'C-Suite' domain. ONLY IF COMPLETE DOMAIN IS COVERED then the key phrase will be in management level. One job title, even if it is on the top or head of the heirarchy, does not cover the complete domain. If a user wants all 'executives', without any business function specified then three management levels will be covered, namely "CSuite," "Executive VP or Sr. VP" and "VP" so all MUST come. However, the word 'executive' is mentioned in relation to a business function, only titles specific to that function should be included. For example, if 'Marketing Executives' is mentioned, titles such as 'CMO', 'Chief Marketing Officer', 'Senior VP of Marketing', 'Senior Vice President of Marketing', 'VP of Marketing', and 'Vice President of Marketing' should be included. The word 'executive' or 'executives' would, thus, NEVER be included neither as job title nor management level.
Job Title Focus: As a CFO would only be a chief in finance (not covering the CSuite domain), the CFO being discussed here comes under job title, not management level, same if the title was "heads of finance" (wouldn't cover Head domain), etc.
Output: {"management_levels": [], "titles": ["CFO", "Chief Finance Officer"]} # Job titles MUST BE CONCISE and TO THE POINT, mentioning ONLY the TITLE and the BUSINESS FUNCTION if the business function is given. No added details, such as company name or group.

If terms like 'leader', 'expert', 'specialist', or similar are mentioned, extract a maximum of 2-3 relevant job titles associated with those terms based on the context, focusing on the most appropriate leadership or expertise roles. However, do not extract titles based on names, companies, etc.

For each management level and title, also tell why you put it there. If a business function can be clearly identified, the key phrase will be a JOB title. ALWAYS make the necessary changes when the word 'executive' or 'leader' or 'expert', etc., is mentioned in the user query and get LOGICAL titles. Management level of 'Manager' will not be chosen when a specific type of 'manager' (senior managers, project manager, etc.) are asked for. ONLY identify and consider complete key phrases EXPLICITLY MENTIONED IN THE PROMPT, and each key phrase will either be in management level or title, NEVER consider THE SAME KEY PHRASE for BOTH. Evaluate key phrases separately.
Always return a JSON object in your output.

User Query: {{QUERY}}
Let's think step by step about each key phrase."""


In [4]:
async def chatgpt_response_a(query, temperature=0.1, model="gpt-4o", **kwargs):

    """
    Function to run prompts on chatgpt

    Args:
        key (string): openai api key
        messages (list): list of object that has the chat that you want to process with chatgpt. i.e. system prompt, assistant prompt and user prompt
        temperature (float, optional): Temperature of gpt for generations. Defaults to 0.7.
        model (str, optional): The model you want to use. Defaults to "gpt-4o".

    Returns:
        string: chatgpt result
    """
    # user_message = Template(NER_MANAGEMENT_LEVEL_TITLE_USER_PROMPT).render({"QUERY" : query})
    user_message = Template(NER_MANAGEMENT_LEVEL_TITLE_USER_PROMPT_ORIGINAL).render({"QUERY" : query})
    messages = [
            # {"role": "system", "content": NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT},
            {"role": "system", "content": NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT_ORIGINAL},
            {"role": "user", "content": f"User Query: {user_message}"},
            
    ]
    # print(messages)
    openai_object = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
    }
    aclient = AsyncOpenAI(api_key=openai_api_key)

    openai_object.update(kwargs)

    response = await aclient.chat.completions.create(**openai_object)
    response = response.__dict__
    response["choices"] = [choice.__dict__ for choice in response["choices"]]
    for choice in response["choices"]:
        choice["message"] = choice["message"].__dict__
    return response

In [5]:
import json
import re

def parse_response(text):
    """
    Parse a text response containing rationale and JSON data, including text after the JSON.
    
    Args:
        text (str): Input text containing rationale, JSON data, and possibly additional text
        
    Returns:
        tuple: (rationale, parsed_json)
            - rationale (str): The explanatory text before and after the JSON combined
            - parsed_json (dict): The parsed JSON data
            
    Example:
        text = "Some explanation\n```json\n{'key': 'value'}\n```\nMore explanation"
        rationale, json_data = parse_response(text)
        # rationale will contain "Some explanation More explanation"
        # json_data will contain {'key': 'value'}
    """
    # Find the JSON part using regex
    # Look for content between ```json and ``` markers
    json_match = re.search(r'```json\s*(.*?)\s*```', text, re.DOTALL)
    
    if not json_match:
        raise ValueError("No JSON content found in the specified format")
    
    # Extract the JSON string and parse it
    json_str = json_match.group(1).strip()
    parsed_json = json.loads(json_str)
    
    # Get the text before the JSON
    text_before = text[:json_match.start()].strip()
    
    # Get the text after the JSON
    text_after = text[json_match.end():].strip()
    
    # Combine the text before and after, with a space between if both exist
    rationale = text_before
    if text_after:
        if rationale:
            rationale += ' '
        rationale += text_after
    
    return rationale, parsed_json

In [134]:
query = "Identify heads of corporate strategy at AI-driven SaaS companies focused on fintech in Europe."

In [135]:
result = await chatgpt_response_a(query)
rationale,result = parse_response(result['choices'][0]['message']['content'])

[{'role': 'system', 'content': '\nYou are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.\n'}, {'role': 'user', 'content': 'User Query: \nInstructions:\n1. Management Levels: Only return management levels that match the predefined set: ["Partners,"," "Founder or Co-founder," "Board of Directors," "CSuite/Chiefs," "Executive VP or Sr. VP," "General Manager," "Manager," "Head," "Senior Partner," "Junior Partner," "VP," "Director," "Senior (All Senior-Level Individual Contributors)," "Mid (All Mid-Level Individual Contributors)," "Junior (All Junior-Level Individual Contributors)"]. MANAGEMENT CAN ONLY BE FROM THIS PREDEFINED SET, Nothing ELSE.\n2. Job Titles: Normalize the job title after extracting it from the text. For example, convert "ceo" to "Chief Executive Officer" and always include both the full title and its abbreviation (confirmed ones), e.g., "VP of Engineering

In [138]:
print('-'*47)
print("Query:")
print(query)
print('-'*47)
print("Rationale:")
print(rationale)
print('-'*47)
print("Result:")
print(result)

-----------------------------------------------
Query:
Identify heads of corporate strategy at AI-driven SaaS companies focused on fintech in Europe.
-----------------------------------------------
Rationale:
To address the user query, we need to identify the key phrases and determine whether they should be classified as management levels or job titles. Let's break down the query:

1. **Key Phrase Identification**: 
   - "heads of corporate strategy"

2. **Classification**:
   - **"heads of corporate strategy"**: This phrase specifies a business function, "corporate strategy," and is associated with the term "heads." Since "heads" is a management level in the predefined set, we need to determine if it covers the complete domain of "Head" in management levels. However, because it specifies a business function, it should be classified as a job title rather than a management level.

3. **Output Construction**:
   - Since "heads of corporate strategy" is classified as a job title, it will 

## Generating Batch of Rationale and Outputs from GPT-4o

In [52]:
list_of_queries = []

In [53]:
for item in data:
    list_of_queries.append(item)

In [54]:
len(list_of_queries)
list_of_queries

['Find human resources experts with proficiency in talent acquisition, employee engagement, strategic performance management, and HRIS software capabilities. Candidates should be based in the vibrant setting of New York City, New York, excluding those with a background in the hospitality industry.',
 'Search for HR professionals skilled in talent recruitment, boosting employee engagement, managing performance metrics, and utilizing HRIS platforms. The candidates must be located in the bustling city of New York, NY, and should not have ties to the hospitality sector.',
 'Identify HR specialists with a strong background in acquiring talent, developing employee engagement strategies, and managing performance, along with expertise in HRIS systems. The search is focused on professionals situated in New York City, NY, with no involvement in the hospitality field.',
 'Locate human resource professionals experienced in talent acquisition, performance management, and HRIS functionalities, who c

In [76]:
tmp = await chatgpt_response_a("Which company is best for technology?", temperature=0.1, model="gpt-4o")

In [78]:
print(tmp['choices'][0]['message']['content'])

In this query, there are no key phrases related to management levels or job titles. The query is asking about companies in the context of technology, but it does not specify any management levels or job titles.

Output: 
```json
{
  "management_levels": [],
  "titles": []
}
```


In [55]:
import asyncio
from typing import List, Dict
import math

async def process_batch(texts: List[str]) -> List[Dict]:
    """Process a single batch of texts using the original title_management logic"""
    ans = []
    tasks = []
    for text in texts:
        tasks.append(chatgpt_response_a(text, temperature=0.1, model="gpt-4o"))

    results = await asyncio.gather(*tasks)
    for query, res in zip(texts, results):
        response = res
        response = response["choices"][0]["message"]["content"]
        rationale, json_output = parse_response(response)
        ans.append({
            "query": query,
            "rationale": rationale,
            "json_output": json_output
        })
    return ans

async def batched_title_management(texts: List[str], batch_size: int = 10) -> List[Dict]:
    """Process texts in batches of specified size"""
    all_results = []
    num_batches = math.ceil(len(texts) / batch_size)
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(texts))
        batch = texts[start_idx:end_idx]
        
        try:
            batch_results = await process_batch(batch)
            all_results.extend(batch_results)
            
            # Optional: Add a small delay between batches to avoid rate limiting
            if i < num_batches - 1:
                await asyncio.sleep(1)
                
        except Exception as e:
            print(f"Error processing batch {i+1}/{num_batches}: {str(e)}")
            # You might want to handle the error differently depending on your needs
            continue
    
    return all_results

In [56]:
async def title_management(texts):
    ans = []
    tasks = []
    for text in texts:
        tasks.append(chatgpt_response_a(text, temperature=0.1, model="gpt-4o"))

    results = await asyncio.gather(*tasks)
    for query, res in zip(texts, results):
        response = res
        response = response["choices"][0]["message"]["content"]
        rationale, json_output = parse_response(response)
        ans.append({
            "query": query,
            "rationale": rationale,
            "json_output": json_output
        })
    return ans

In [57]:
len(list_of_queries)

70

In [58]:
results = await batched_title_management(list_of_queries, batch_size=10)

In [79]:
len(results)

70

In [59]:
# with open("training_data_dicts_version2.json", "w") as f:
#     json.dump(results, f, indent=2)

In [80]:
results = []
with open("training_data_dicts_version2.json") as f:
    results = json.load(f)

In [81]:
len(results)

51

## Creating FineTuning Dataset for smaller model

In [82]:
STUDENT_MODEL_PROMPT_SYSTEM = """You are an AI assistant tasked with extracting management levels and job titles from a given query. Your goal is to analyze the query, identify relevant key phrases, and categorize them appropriately as either management levels or job titles while generating proper reasoning and thinking step by step. 
Your output format should be like below:
<rationale>
Provide reasoning and step-by-step thinking inside here
</rationale>
<json_output>
Provide json output inside here
</json_output>
"""

STUDENT_MODEL_PROMPT_USER_V3 = """
Instructions:
1. Identify key phrases that are management levels or job titles.
2. Categorize each key phrase as either a management level or a job title.
3. Use predefined management levels: ['Partners,',' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senior (All Senior-Level Individual Contributors),' 'Mid (All Mid-Level Individual Contributors),' 'Junior (All Junior-Level Individual Contributors)'].
4. For job titles:
   - Normalize and include abbreviations if applicable.
   - Exclude company or region names.
   
Always return a JSON object. 


Here is the query you need to analyze: `{{QUERY}}`
Let's think step by step about each key phrase.
"""




ASSISTANT_RESPONSE_TEMPLATE = """
<rationale>
{{rationale}}
</rationale>

<json_output>
{{json_output}}
</json_output>
"""


In [212]:
with open('generated_input_output_pairs_v2.json') as f:
    generated_input_output_pairs = json.load(f)

In [213]:
len(generated_input_output_pairs)

423

In [83]:
example = results[0]

In [84]:
def generate_training_example(example):
    user_message = Template(STUDENT_MODEL_PROMPT_USER_V3).render({"QUERY": example["query"]})
    assistant_response = Template(ASSISTANT_RESPONSE_TEMPLATE).render({"rationale": example["rationale"], "json_output": example["json_output"]})
    
    messages = [
            {"role": "system", "content": STUDENT_MODEL_PROMPT_SYSTEM},
            {"role": "user", "content": f"""User Query: {user_message}"""},
            {"role": "assistant", "content": assistant_response}       
    ]
    return { "messages" : messages }
    

In [85]:
training_example1 = generate_training_example(example)

In [86]:
training_example1

{'messages': [{'role': 'system',
   'content': 'You are an AI assistant tasked with extracting management levels and job titles from a given query. Your goal is to analyze the query, identify relevant key phrases, and categorize them appropriately as either management levels or job titles while generating proper reasoning and thinking step by step. \nYour output format should be like below:\n<rationale>\nProvide reasoning and step-by-step thinking inside here\n</rationale>\n<json_output>\nProvide json output inside here\n</json_output>\n'},
  {'role': 'user',
   'content': "User Query: \nInstructions:\n1. Identify key phrases that are management levels or job titles.\n2. Categorize each key phrase as either a management level or a job title.\n3. Use predefined management levels: ['Partners,',' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senior (All 

In [87]:
openai_finetuning_data_json = []

In [189]:
# with open("openai_gpt4o_mini_finetuning_data.jsonl", "w") as f:
#     for item in generated_input_output_pairs:
#         training_example = generate_training_example(item)
#         f.write((training_example) + "\n")


In [88]:
for item in results:
    training_example = generate_training_example(item)
    openai_finetuning_data_json.append((training_example))

In [89]:
openai_finetuning_data_json[0]

{'messages': [{'role': 'system',
   'content': 'You are an AI assistant tasked with extracting management levels and job titles from a given query. Your goal is to analyze the query, identify relevant key phrases, and categorize them appropriately as either management levels or job titles while generating proper reasoning and thinking step by step. \nYour output format should be like below:\n<rationale>\nProvide reasoning and step-by-step thinking inside here\n</rationale>\n<json_output>\nProvide json output inside here\n</json_output>\n'},
  {'role': 'user',
   'content': "User Query: \nInstructions:\n1. Identify key phrases that are management levels or job titles.\n2. Categorize each key phrase as either a management level or a job title.\n3. Use predefined management levels: ['Partners,',' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senior (All 

In [90]:
import random

# Seed for reproducibility
seed = 42
random.seed(seed)

# Shuffle the data
random.shuffle(openai_finetuning_data_json)

# Define split ratios
train_ratio = 0.7
eval_ratio = 0.16  # Remaining 20% will automatically be test_ratio

# Calculate split indices
total_size = len(openai_finetuning_data_json)
train_size = int(total_size * train_ratio)
eval_size = int(total_size * eval_ratio)

# Split the data
train_data = openai_finetuning_data_json[:train_size]
eval_data = openai_finetuning_data_json[train_size:train_size + eval_size]
test_data = openai_finetuning_data_json[train_size + eval_size:]

# Output
print("Train Set:", len(train_data), train_data[:5])  # Showing first 5 for brevity
print("Eval Set:", len(eval_data), eval_data[:5])
print("Test Set:", len(test_data), test_data[:5])


Train Set: 35 [{'messages': [{'role': 'system', 'content': 'You are an AI assistant tasked with extracting management levels and job titles from a given query. Your goal is to analyze the query, identify relevant key phrases, and categorize them appropriately as either management levels or job titles while generating proper reasoning and thinking step by step. \nYour output format should be like below:\n<rationale>\nProvide reasoning and step-by-step thinking inside here\n</rationale>\n<json_output>\nProvide json output inside here\n</json_output>\n'}, {'role': 'user', 'content': "User Query: \nInstructions:\n1. Identify key phrases that are management levels or job titles.\n2. Categorize each key phrase as either a management level or a job title.\n3. Use predefined management levels: ['Partners,',' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senio

In [197]:
# with open("train.json", "w") as f:
#     json.dump(train_data, f, indent=2)
    
# with open("test.json", "w") as f:
#     json.dump(test_data, f, indent=2)

# with open("eval.json", "w") as f:
#     json.dump(eval_data, f, indent=2)


In [91]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [92]:
write_jsonl(train_data, "train_data_v2.jsonl")
write_jsonl(eval_data, "eval_data_v2.jsonl")
write_jsonl(test_data, "test_data_v2.jsonl")

In [216]:
"""["Partners,"," "Founder or Co-founder," "Board of Directors," "CSuite/Chiefs," "Executive VP or Sr. VP," "General Manager," "Manager," "Head," "Senior Partner," "Junior Partner," "VP," "Director," "Senior (All Senior-Level Individual Contributors)," "Mid (All Mid-Level Individual Contributors)," "Junior (All Junior-Level Individual Contributors)"].""".replace('"', "'")

"['Partners,',' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senior (All Senior-Level Individual Contributors),' 'Mid (All Mid-Level Individual Contributors),' 'Junior (All Junior-Level Individual Contributors)']."